# Importing Pandas 

In [1]:
import pandas as pd

In [2]:
import scipy.stats as stats #for mode

# Functions

In [4]:
#extract year and date from time
def extract_year_date(df): 
    df['time'] = pd.to_datetime(df['time'])
    df['year'] =df['time'].dt.year
    df['date']= df['time'].dt.date #used for aggregating weather parameters (resampling())
    return df

In [5]:
#convert degree to cardinal direction
def degrees_to_cardinal(angle): 
    directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
    ix = int((angle + 11.25) /22.5) 
    return directions[ix%16]

In [6]:
#aggregating wind direction
def aggregate_wind_dir(df):
    df['wind_dir_text'] = df['winddirection_10m (°)'].map(lambda x: degrees_to_cardinal(x)) #convert degrees to cardinal direction
    wind_dir = df.groupby("year")['wind_dir_text'].agg(lambda x:x.value_counts().index[0]) #get the most frequent direction per year
    return wind_dir

In [7]:
#get most frequent wind_speed
def get_wind_speed(wind_speed,wind_dir): 
    windSpeed = []
    for idx,val in enumerate(wind_dir): 
        windSpeed.append(wind_speed[(wind_speed['year']== wind_dir.index[idx]) & (wind_speed['wind_dir_text']==val)]['windspeed_10m (km/h)'].item())
    return windSpeed

In [8]:
#aggregating wind speed
def aggregate_wind_speed(df,wind_dir):
    #get the index of the most frequent speed per year&direction 
    wind_speed = df.groupby(['year','wind_dir_text'])['windspeed_10m (km/h)'].apply(lambda x: x.value_counts().idxmax()) 
    return get_wind_speed(wind_speed.reset_index(), wind_dir)

In [9]:
#put aggregated wind speed and wind direction in a single dataframe
def rescaled_wind(dir,speed):
    wind = pd.DataFrame({'year':dir.index,'Wind_Direction': dir.values})
    wind['Wind_Speed_(km/h)'] = speed
    return wind

### Aggregating remaining weather parameters

In [10]:
def get_mean(df):
    ave_cols = df[['date','temperature_2m (°C)', 'relativehumidity_2m (%)', 'dewpoint_2m (°C)','surface_pressure (hPa)','et0_fao_evapotranspiration (mm)', 
'vapor_pressure_deficit (kPa)','soil_temperature_28_to_100cm (°C)','soil_moisture_28_to_100cm (m³/m³)','direct_radiation (W/m²)','diffuse_radiation (W/m²)']]
    ave_cols.set_index('date',inplace = True) #set index as date
    ave_cols.index = pd.to_datetime(ave_cols.index)
    ave_cols = ave_cols.resample('A').mean()
    return ave_cols.round(2)

In [11]:
def get_sum(df):
    sum_cols = df[['date','rain (mm)']]
    sum_cols.set_index('date',inplace = True) #set index as date
    sum_cols.index = pd.to_datetime(sum_cols.index)
    sum_cols = sum_cols.resample('A').sum()
    return sum_cols.round(2) 

In [12]:
def get_mode(df): 
    mode_cols = df[['date','year','weathercode (wmo code)','windgusts_10m (km/h)', 'cloudcover (%)']]
    mode_cols.set_index('date',inplace = True) 
    mode_cols.index = pd.to_datetime(mode_cols.index)
    mode_cols = mode_cols.resample('A').apply(lambda x: stats.mode(x)[0])
    return mode_cols

### Aggregating data for regions with more farms > 1:  

In [13]:
def region_mode(df):
    annual_mode = df.groupby("year")[['Wind_Direction','weathercode (wmo code)','windgusts_10m (km/h)', 'cloudcover (%)']].agg(lambda x:x.value_counts().index[0]) #get the most frequent direction per year
    return annual_mode

In [14]:
#aggregating wind speed for region
def aggregate_region_windSpeed(df, region_mode): 
    wind_speed = df.groupby(['year','Wind_Direction'])['Wind_Speed_(km/h)'].apply(lambda x: x.value_counts().idxmax()) #get the most frequent speed per year&direction
    wind_speed = region_wind_speed(wind_speed.reset_index(), region_mode)
    region_mode['Wind_Speed_(km/h)'] = wind_speed
    return region_mode.reset_index()

In [15]:
#get the most frequent speed per year & most frequent direction
def region_wind_speed(wind_speed,region_mode):
    windSpeed  = []
    for index,row in region_mode.iterrows(): 
        windSpeed.append(wind_speed[(wind_speed['year']== index) & (wind_speed['Wind_Direction']==row['Wind_Direction'])]['Wind_Speed_(km/h)'].item())
    return windSpeed

In [16]:
def region_mean(df):
    annual_mean = df.groupby("year")[['temperature_2m (°C)',
       'relativehumidity_2m (%)', 'dewpoint_2m (°C)', 'surface_pressure (hPa)',
       'et0_fao_evapotranspiration (mm)', 'vapor_pressure_deficit (kPa)',
       'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_28_to_100cm (m³/m³)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)', 'rain (mm)']].mean() #get the most frequent direction per year
    return annual_mean.reset_index()

### Steps in aggregating
1. Convert time column to datetime
2. Extract year - used for aggregating wind parameters
3. Extract date - used for aggregating non-wind parameters
4. Aggregate wind direction 
    * convert direction in degrees to cardinal direction 
    * get the mode for each year  
5. Aggregate wind speed 
    * Get the most frequent wind speed given the mode wind direction of that year. 
    Ex: get the most frequent wind speed for 1990 where the direction is the mode direction - Northeast (NE)
6. Merge wind direction and speed dataframes to serve as the aggregate dataset for wind parameters
7. Aggregate temperature, relative humdiity, dewpoint,surface pressure, evapotranspiration, vapor pressure, soil columns, and radiation columns using mean
8. Aggregate rain using sum 
9. Aggregate weathercode, cloud cover, and wind gust using mode
10. Merge all aggregated non-wind columns (including wind gusts) to serve as the aggregate dataset for non-wind parameters
11. Merge wind and non-wind dataframe to serve as aggregate weather dataset for one location

# Region 9 - Zamboanga Peninsula

## Zamboanga del Norte

#### Atoaa Coffee Farm

In [16]:
zamboanga_norte_1 = pd.read_csv(r"D:\Thesis\Datasets\9-Zamboanga del Norte\ATOAA Coffee Farm.csv",header = 2) 

In [17]:
zamboanga_norte_1.head()

,time,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),rain (mm),weathercode (wmo code),surface_pressure (hPa),cloudcover (%),et0_fao_evapotranspiration (mm),vapor_pressure_deficit (kPa),windspeed_10m (km/h),winddirection_10m (°),windgusts_10m (km/h),soil_temperature_28_to_100cm (°C),soil_moisture_28_to_100cm (m³/m³),direct_radiation (W/m²),diffuse_radiation (W/m²)
0,1990-01-01T00:00,24.7,82,21.5,0.0,2,1009.8,51,0.04,0.55,17.3,54,31.3,27.1,0.408,0.0,0.0
1,1990-01-01T01:00,24.5,84,21.6,0.1,51,1009.5,35,0.03,0.49,17.2,53,29.9,27.1,0.408,0.0,0.0
2,1990-01-01T02:00,24.3,85,21.7,0.2,51,1009.0,57,0.03,0.45,16.5,54,28.4,27.1,0.408,0.0,0.0
3,1990-01-01T03:00,24.5,85,21.7,0.2,51,1008.7,65,0.02,0.47,15.4,53,25.9,27.1,0.408,0.0,0.0
4,1990-01-01T04:00,24.3,86,21.8,0.3,51,1008.5,45,0.02,0.43,16.4,50,25.9,27.0,0.408,0.0,0.0


In [18]:
zamboanga_norte_1 = extract_year_date(zamboanga_norte_1)

#### Aggregate wind parameters

In [19]:
zamboanga_norte_1_dir = aggregate_wind_dir(zamboanga_norte_1) 

In [20]:
zamboanga_norte_1_speed = aggregate_wind_speed(zamboanga_norte_1,zamboanga_norte_1_dir)

In [21]:
zamboanga_norte_1_wind= rescaled_wind(zamboanga_norte_1_dir,zamboanga_norte_1_speed)

#### Aggregating non wind parameters

In [22]:
zamboanga_norte_1_ave = get_mean(zamboanga_norte_1)

In [23]:
zamboanga_norte_1_sum = get_sum(zamboanga_norte_1)

In [24]:
zamboanga_norte_1_mode = get_mode(zamboanga_norte_1)

In [25]:
zamboanga_norte_1_not_wind = pd.concat([zamboanga_norte_1_ave, zamboanga_norte_1_sum,zamboanga_norte_1_mode],axis = 1)

In [26]:
zamboanga_norte_1_final = zamboanga_norte_1_wind.merge(zamboanga_norte_1_not_wind , how='inner',on='year')

#### Good Morning Farm

In [27]:
zamboanga_norte_2 = pd.read_csv(r"D:\Thesis\Datasets\9-Zamboanga del Norte\Good Morning Coffee.csv",header = 2) 

In [28]:
zamboanga_norte_2 = extract_year_date(zamboanga_norte_2)

In [29]:
zamboanga_norte_2_dir = aggregate_wind_dir(zamboanga_norte_2) 

In [30]:
zamboanga_norte_2_speed = aggregate_wind_speed(zamboanga_norte_2 ,zamboanga_norte_2_dir)

In [31]:
zamboanga_norte_2_wind= rescaled_wind(zamboanga_norte_2_dir,zamboanga_norte_2_speed)

In [32]:
zamboanga_norte_2_ave = get_mean(zamboanga_norte_2)

In [33]:
zamboanga_norte_2_sum = get_sum(zamboanga_norte_2)

In [34]:
zamboanga_norte_2_mode = get_mode(zamboanga_norte_2)

In [35]:
zamboanga_norte_2_not_wind = pd.concat([zamboanga_norte_2_ave, zamboanga_norte_2_sum,zamboanga_norte_2_mode],axis = 1)

In [36]:
zamboanga_norte_2_final = zamboanga_norte_2_wind.merge(zamboanga_norte_2_not_wind , how='inner',on='year')

#### Get aggregate for Zamboanga del Norte given two locations

In [37]:
zamboanga_norte = pd.concat([zamboanga_norte_1_final,zamboanga_norte_2_final],axis = 0)

In [114]:
regional_mode = region_mode(zamboanga_norte)

In [116]:
regional_mode = aggregate_region_windSpeed(zamboanga_norte,regional_mode)

In [127]:
regional_mean = region_mean(zamboanga_norte)

In [128]:
zamboanga_norte_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [129]:
zamboanga_norte_annual.insert(loc = 1, column= "Location", value ="ZAMBOANGA DEL NORTE")

In [55]:
zamboanga_norte_annual.to_csv('ZAMBOANGA DEL NORTE.csv')

## Zamboanga del Sur 

#### Dumalinao

In [57]:
zamboanga_sur_1 = pd.read_csv(r"D:\Thesis\Datasets\9 - Zamboanga del Sur\Dumalinao, Zamboanga del Sur.csv",header = 2) 

In [59]:
zamboanga_sur_1 = extract_year_date(zamboanga_sur_1)

In [60]:
zamboanga_sur_1_dir = aggregate_wind_dir(zamboanga_sur_1) 

In [61]:
zamboanga_sur_1_speed = aggregate_wind_speed(zamboanga_sur_1,zamboanga_sur_1_dir)

In [62]:
zamboanga_sur_1_wind = rescaled_wind(zamboanga_sur_1_dir,zamboanga_sur_1_speed)

In [63]:
zamboanga_sur_1_ave = get_mean(zamboanga_sur_1)

In [64]:
zamboanga_sur_1_sum = get_sum(zamboanga_sur_1)

In [65]:
zamboanga_sur_1_mode = get_mode(zamboanga_sur_1)

In [69]:
zamboanga_sur_1_not_wind = pd.concat([zamboanga_sur_1_ave,zamboanga_sur_1_sum,zamboanga_sur_1_mode],axis = 1)

In [71]:
zamboanga_sur_1_final = zamboanga_sur_1_wind.merge(zamboanga_sur_1_not_wind, how = 'inner', on='year')

#### Felicisimo Coffee

In [73]:
zamboanga_sur_2 = pd.read_csv(r"D:\Thesis\Datasets\9 - Zamboanga del Sur\Felicisimo Coffee.csv",header = 2) 

In [74]:
zamboanga_sur_2= extract_year_date(zamboanga_sur_2)

In [75]:
zamboanga_sur_2_dir = aggregate_wind_dir(zamboanga_sur_2) 

In [76]:
zamboanga_sur_2_speed = aggregate_wind_speed(zamboanga_sur_2,zamboanga_sur_2_dir)

In [77]:
zamboanga_sur_2_wind = rescaled_wind(zamboanga_sur_2_dir,zamboanga_sur_2_speed)

In [78]:
zamboanga_sur_2_ave = get_mean(zamboanga_sur_2)

In [79]:
zamboanga_sur_2_sum = get_sum(zamboanga_sur_2)

In [80]:
zamboanga_sur_2_mode = get_mode(zamboanga_sur_2)

In [81]:
zamboanga_sur_2_not_wind = pd.concat([zamboanga_sur_2_ave,zamboanga_sur_2_sum,zamboanga_sur_2_mode],axis = 1)

In [82]:
zamboanga_sur_2_final = zamboanga_sur_1_wind.merge(zamboanga_sur_2_not_wind, how='inner',on='year')

#### For Zamboanga del Sur

In [87]:
zamboanga_sur = pd.concat([zamboanga_sur_1_final,zamboanga_sur_2_final],axis = 0)

In [121]:
regional_mode = region_mode(zamboanga_sur)

In [123]:
regional_mode = aggregate_region_windSpeed(zamboanga_sur,regional_mode)

In [130]:
regional_mean = region_mean(zamboanga_sur)

In [131]:
zamboanga_sur_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [132]:
zamboanga_sur_annual.insert(loc = 1, column= "Location", value = 'ZAMBOANGA DEL SUR')

In [133]:
zamboanga_sur_annual.head()

,year,Location,Wind_Direction,weathercode (wmo code),windgusts_10m (km/h),cloudcover (%),Wind_Speed_(km/h),temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),surface_pressure (hPa),et0_fao_evapotranspiration (mm),vapor_pressure_deficit (kPa),soil_temperature_28_to_100cm (°C),soil_moisture_28_to_100cm (m³/m³),direct_radiation (W/m²),diffuse_radiation (W/m²),rain (mm)
0,1990,ZAMBOANGA DEL SUR,NNE,1,10.8,100,6.0,26.330,81.795,22.850,996.970,0.16,0.650,28.420,0.395,148.220,70.510,2353.10
1,1991,ZAMBOANGA DEL SUR,NNE,1,9.0,100,5.8,26.205,81.790,22.740,997.210,0.16,0.640,28.255,0.390,145.260,74.580,1834.60
2,1992,ZAMBOANGA DEL SUR,NNE,1,8.3,100,7.6,26.135,81.855,22.695,997.500,0.16,0.640,28.315,0.385,144.765,74.865,1927.00
3,1993,ZAMBOANGA DEL SUR,NNE,1,9.4,100,5.2,26.005,83.105,22.830,997.365,0.15,0.590,27.820,0.400,138.955,75.470,2235.85
4,1994,ZAMBOANGA DEL SUR,NNE,1,10.8,100,6.6,25.935,83.705,22.885,997.170,0.15,0.565,27.525,0.420,137.640,73.300,2392.85


In [134]:
zamboanga_sur_annual.to_csv('ZAMBOANGA DEL SUR.csv')

## Zamboanga Sibugay

In [135]:
zamboanga_sibugay_1 = pd.read_csv(r"D:\Thesis\Datasets\9 - Zamboanga Sibugay\Buug, Zamboanga Sibugay.csv",header = 2) 

In [136]:
zamboanga_sibugay_1 = extract_year_date(zamboanga_sibugay_1)

In [137]:
zamboanga_sibugay_1_dir = aggregate_wind_dir(zamboanga_sibugay_1)

In [139]:
zamboanga_sibugay_1_speed = aggregate_wind_speed(zamboanga_sibugay_1,zamboanga_sibugay_1_dir)

In [140]:
zamboanga_sibugay_1_wind= rescaled_wind(zamboanga_sibugay_1_dir,zamboanga_sibugay_1_speed )

In [141]:
zamboanga_sibugay_1_ave = get_mean(zamboanga_sibugay_1)

In [142]:
zamboanga_sibugay_1_sum = get_sum(zamboanga_sibugay_1)

In [143]:
zamboanga_sibugay_1_mode = get_mode(zamboanga_sibugay_1)

In [144]:
zamboanga_sibugay_1_not_wind = pd.concat([zamboanga_sibugay_1_ave,zamboanga_sibugay_1_sum,zamboanga_sibugay_1_mode ],axis = 1)

In [145]:
zamboanga_sibugay_1_final = zamboanga_sibugay_1_wind.merge(zamboanga_sibugay_1_not_wind, how='inner',on='year')

In [147]:
zamboanga_sibugay_1_final.insert(loc = 1, column= "Location", value = "ZAMBOANGA SIBUGAY")

In [150]:
zamboanga_sibugay_1_final.to_csv("ZAMBOANGA SIBUGAY.csv")

## Zamboanga City

# Region 10 


## Bukidnon

#### Bayanihan Millennim MultiPurpose

In [166]:
bukidnon_1 = pd.read_csv(r"D:\Thesis\Datasets\10 - Bukidnon\Bayanihan Millennim MultiPurpose.csv",header = 2) 

In [167]:
bukidnon_1 = extract_year_date(bukidnon_1)

In [173]:
bukidnon_1_dir = aggregate_wind_dir(bukidnon_1)

In [174]:
bukidnon_1_speed = aggregate_wind_speed(bukidnon_1,bukidnon_1_dir)

In [175]:
bukidnon_1_wind= rescaled_wind(bukidnon_1_dir,bukidnon_1_speed)

In [177]:
bukidnon_1_ave = get_mean(bukidnon_1)

In [178]:
bukidnon_1_sum = get_sum(bukidnon_1)

In [179]:
bukdinon_1_mode = get_mode(bukidnon_1)

In [180]:
bukidnon_1_not_wind = pd.concat([bukidnon_1_ave, bukidnon_1_sum,bukdinon_1_mode],axis = 1)

In [181]:
bukidnon_1_final = bukidnon_1_wind.merge(bukidnon_1_not_wind , how='inner',on='year')

#### Cora Llauderes Imdalsa

In [182]:
bukidnon_2 = pd.read_csv(r"D:\Thesis\Datasets\10 - Bukidnon\Cora Llauderes Imdalsa.csv",header = 2) 

In [183]:
bukidnon_2 = extract_year_date(bukidnon_2)

In [184]:
bukidnon_2_dir = aggregate_wind_dir(bukidnon_2)

In [191]:
bukidnon_2_speed = aggregate_wind_speed(bukidnon_2,bukidnon_2_dir)

In [192]:
bukidnon_2_wind= rescaled_wind(bukidnon_2_dir,bukidnon_2_speed)

In [195]:
bukidnon_2_ave = get_mean(bukidnon_2)

In [194]:
bukidnon_2_sum = get_sum(bukidnon_2)

In [196]:
bukidnon_2_mode = get_mode(bukidnon_2)

In [197]:
bukidnon_2_not_wind = pd.concat([bukidnon_2_ave, bukidnon_2_sum,bukidnon_2_mode],axis = 1)

In [198]:
bukidnon_2_final = bukidnon_2_wind.merge(bukidnon_2_not_wind , how='inner',on='year')

#### Kape Maramag

In [199]:
bukidnon_3 = pd.read_csv(r"D:\Thesis\Datasets\10 - Bukidnon\Kape Maramag.csv",header = 2) 

In [201]:
bukidnon_3= extract_year_date(bukidnon_3)

In [202]:
bukidnon_3_dir = aggregate_wind_dir(bukidnon_3)

In [203]:
bukidnon_3_speed = aggregate_wind_speed(bukidnon_3,bukidnon_3_dir)

In [204]:
bukidnon_3_wind= rescaled_wind(bukidnon_3_dir,bukidnon_3_speed)

In [205]:
bukidnon_3_ave = get_mean(bukidnon_3)

In [206]:
bukidnon_3_sum = get_sum(bukidnon_3)

In [207]:
bukidnon_3_mode = get_mode(bukidnon_3)

In [208]:
bukidnon_3_not_wind = pd.concat([bukidnon_3_ave, bukidnon_3_sum,bukidnon_3_mode],axis = 1)

In [209]:
bukidnon_3_final = bukidnon_3_wind.merge(bukidnon_3_not_wind , how='inner',on='year')

#### Imeldas Farm

In [220]:
bukidnon_4 = pd.read_csv(r"D:\Thesis\Datasets\10 - Bukidnon\Imeldas Farm.csv",header = 2) 

In [221]:
bukidnon_4= extract_year_date(bukidnon_4)

In [222]:
bukidnon_4_dir = aggregate_wind_dir(bukidnon_4)

In [223]:
bukidnon_4_speed = aggregate_wind_speed(bukidnon_4,bukidnon_4_dir)

In [224]:
bukidnon_4_wind= rescaled_wind(bukidnon_4_dir,bukidnon_4_speed)

In [225]:
bukidnon_4_ave = get_mean(bukidnon_4)

In [226]:
bukidnon_4_sum = get_sum(bukidnon_4)

In [227]:
bukidnon_4_mode = get_mode(bukidnon_4)

In [228]:
bukidnon_4_not_wind = pd.concat([bukidnon_4_ave, bukidnon_4_sum,bukidnon_4_mode],axis = 1)

In [229]:
bukidnon_4_final = bukidnon_4_wind.merge(bukidnon_4_not_wind , how='inner',on='year')

#### Kumaykay Riverfarm 

In [230]:
bukidnon_5 = pd.read_csv(r"D:\Thesis\Datasets\10 - Bukidnon\Kumaykay Riverfarm.csv",header = 2) 

In [233]:
bukidnon_5= extract_year_date(bukidnon_5)

In [234]:
bukidnon_5_dir = aggregate_wind_dir(bukidnon_5)

In [235]:
bukidnon_5_speed = aggregate_wind_speed(bukidnon_5,bukidnon_5_dir)

In [236]:
bukidnon_5_wind= rescaled_wind(bukidnon_5_dir,bukidnon_5_speed)

In [237]:
bukidnon_5_ave= get_mean(bukidnon_5)

In [238]:
bukidnon_5_sum = get_sum(bukidnon_5)

In [239]:
bukidnon_5_mode = get_mode(bukidnon_5)

In [240]:
bukidnon_5_not_wind = pd.concat([bukidnon_5_ave, bukidnon_5_sum,bukidnon_5_mode],axis = 1)

In [241]:
bukidnon_5_final = bukidnon_5_wind.merge(bukidnon_5_not_wind , how='inner',on='year')

#### Zambranos Integrated Farm

In [242]:
bukidnon_6 = pd.read_csv(r"D:\Thesis\Datasets\10 - Bukidnon\Zambranos Integrated Farm.csv",header = 2) 

In [243]:
bukidnon_6= extract_year_date(bukidnon_6)

In [244]:
bukidnon_6_dir = aggregate_wind_dir(bukidnon_6)

In [245]:
bukidnon_6_speed = aggregate_wind_speed(bukidnon_6,bukidnon_6_dir)

In [246]:
bukidnon_6_wind= rescaled_wind(bukidnon_6_dir,bukidnon_6_speed)

In [247]:
bukidnon_6_ave = get_mean(bukidnon_6)

In [248]:
bukidnon_6_sum = get_sum(bukidnon_6)

In [249]:
bukidnon_6_mode = get_mode(bukidnon_6)

In [250]:
bukidnon_6_not_wind = pd.concat([bukidnon_6_ave, bukidnon_6_sum,bukidnon_6_mode],axis = 1)

In [251]:
bukidnon_6_final = bukidnon_6_wind.merge(bukidnon_6_not_wind , how='inner',on='year')

#### For Bukidnon

In [252]:
bukidnon = pd.concat([bukidnon_1_final, bukidnon_2_final, bukidnon_3_final, bukidnon_4_final, 
                      bukidnon_5_final, bukidnon_6_final],axis = 0)

In [253]:
regional_mode = region_mode(bukidnon)

In [254]:
regional_mode = aggregate_region_windSpeed(bukidnon,regional_mode)

In [255]:
regional_mean = region_mean(bukidnon)

In [256]:
bukidnon_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [257]:
bukidnon_annual.insert(loc = 1, column= "Location", value = 'BUKIDNON')

In [258]:
bukidnon_annual.to_csv('BUKIDNON.csv')

## Camiguin

In [266]:
camiguin_1= pd.read_csv(r"D:\Thesis\Datasets\10 - Camiguin\Alga,Catarman,Camiguin.csv",header = 2) 

In [268]:
camiguin_1= extract_year_date(camiguin_1)

In [270]:
camiguin_1_dir = aggregate_wind_dir(camiguin_1)

In [271]:
camiguin_1_speed = aggregate_wind_speed(camiguin_1,camiguin_1_dir)

In [272]:
camiguin_1_wind= rescaled_wind(camiguin_1_dir,camiguin_1_speed)

In [273]:
camiguin_1_ave = get_mean(camiguin_1)

In [274]:
camiguin_1_sum = get_sum(camiguin_1)

In [275]:
camiguin_1_mode = get_mode(camiguin_1)

In [276]:
camiguin_1_not_wind = pd.concat([camiguin_1_ave, camiguin_1_sum,camiguin_1_mode],axis = 1)

In [277]:
camiguin_1_final = camiguin_1_wind.merge(camiguin_1_not_wind , how='inner',on='year')

In [278]:
camiguin_1_final.insert(loc = 1, column= "Location", value = "CAMIGUIN")

In [279]:
camiguin_1_final.to_csv("CAMIGUIN.csv")

## Lanao del Norte

#### Sikyop Agriculture Cooperative (SAC)

In [386]:
lanao_norte_1= pd.read_csv(r"D:\Thesis\Datasets\10 - Lanao del Norte\Sikyop Agriculture Cooperative (SAC).csv",header = 2) 

In [387]:
lanao_norte_1= extract_year_date(lanao_norte_1)

In [388]:
lanao_norte_1_dir = aggregate_wind_dir(lanao_norte_1)

In [389]:
lanao_norte_1_speed = aggregate_wind_speed(lanao_norte_1,lanao_norte_1_dir)

In [390]:
lanao_norte_1_wind= rescaled_wind(lanao_norte_1_dir,lanao_norte_1_speed)

In [391]:
lanao_norte_1_ave = get_mean(lanao_norte_1)

In [392]:
lanao_norte_1_sum = get_sum(lanao_norte_1)

In [393]:
lanao_norte_1_mode = get_mode(lanao_norte_1)

In [394]:
lanao_norte_1_not_wind = pd.concat([lanao_norte_1_ave,lanao_norte_1_sum,lanao_norte_1_mode],axis = 1)

In [395]:
lanao_norte_1_final = lanao_norte_1_wind.merge(lanao_norte_1_not_wind , how='inner',on='year')

In [396]:
lanao_norte_1_final.insert(loc = 1, column= "Location", value = "LANAO DEL NORTE")

In [397]:
lanao_norte_1_final.to_csv("LANAO DEL NORTE.csv")

## Misamis Occidental

In [92]:
mis_oc_1= pd.read_csv(r"D:\Thesis\Datasets\10 - Misamis Occidental\vimas_agriventure.csv",header = 2) 

In [93]:
mis_oc_1= extract_year_date(mis_oc_1)

In [94]:
mis_oc_1_dir = aggregate_wind_dir(mis_oc_1)

In [96]:
mis_oc_1_speed = aggregate_wind_speed(mis_oc_1,mis_oc_1_dir )

In [97]:
mis_oc_1_wind= rescaled_wind(mis_oc_1_dir,mis_oc_1_speed)

In [98]:
mis_oc_1_ave = get_mean(mis_oc_1)

In [99]:
mis_oc_1_sum = get_sum(mis_oc_1)

In [100]:
mis_oc_1_mode = get_mode(mis_oc_1)

In [101]:
mis_oc_1_not_wind = pd.concat([mis_oc_1_ave, mis_oc_1_sum,mis_oc_1_mode],axis = 1)

In [102]:
mis_oc_1_final = mis_oc_1_wind.merge(mis_oc_1_not_wind , how='inner',on='year')

In [103]:
mis_oc_1_final.insert(loc = 1, column= "Location", value = "MISAMIS OCCIDENTAL")

In [104]:
mis_oc_1_final.to_csv("MISAMIS OCCIDENTAL.csv")

## Misamis Oriental

#### Kape Mindanaw

In [295]:
mis_or_1= pd.read_csv(r"D:\Thesis\Datasets\10 - Misamis Oriental\KPMFIKAPE MINDANAW.csv",header = 2) 

In [296]:
mis_or_1= extract_year_date(mis_or_1)

In [297]:
mis_or_1_dir = aggregate_wind_dir(mis_or_1)

In [298]:
mis_or_1_speed = aggregate_wind_speed(mis_or_1,mis_or_1_dir)

In [299]:
mis_or_1_wind= rescaled_wind(mis_or_1_dir,mis_or_1_speed)

In [300]:
mis_or_1_ave = get_mean(mis_or_1)

In [301]:
mis_or_1_sum = get_sum(mis_or_1)

In [302]:
mis_or_1_mode = get_mode(mis_or_1)

In [303]:
mis_or_1_not_wind = pd.concat([mis_or_1_ave, mis_or_1_sum,mis_or_1_mode],axis = 1)

In [304]:
mis_or_1_final = mis_or_1_wind.merge(mis_or_1_not_wind , how='inner',on='year')

#### Kape Claveria Beverage Manufacturing

In [305]:
mis_or_2= pd.read_csv(r"D:\Thesis\Datasets\10 - Misamis Oriental\Kape Claveria Beverage Manufacturing.csv",header = 2) 

In [306]:
mis_or_2= extract_year_date(mis_or_2)

In [307]:
mis_or_2_dir = aggregate_wind_dir(mis_or_2)

In [308]:
mis_or_2_speed = aggregate_wind_speed(mis_or_2,mis_or_2_dir)

In [309]:
mis_or_2_wind= rescaled_wind(mis_or_2_dir,mis_or_2_speed)

In [310]:
mis_or_2_ave = get_mean(mis_or_2)

In [311]:
mis_or_2_sum = get_sum(mis_or_2)

In [312]:
mis_or_2_mode = get_mode(mis_or_2)

In [313]:
mis_or_2_not_wind = pd.concat([mis_or_2_ave,mis_or_2_sum,mis_or_2_mode],axis = 1)

In [314]:
mis_or_2_final = mis_or_2_wind.merge(mis_or_2_not_wind , how='inner',on='year')

#### For Misamis Oriental

In [315]:
mis_or = pd.concat([mis_or_1_final,mis_or_2_final],axis = 0)

In [316]:
regional_mode = region_mode(mis_or)

In [317]:
regional_mode = aggregate_region_windSpeed(mis_or,regional_mode)

In [318]:
regional_mean = region_mean(mis_or)

In [319]:
mis_or_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [320]:
mis_or_annual.insert(loc = 1, column= "Location", value = 'MISAMIS ORIENTAL')

In [321]:
mis_or_annual.to_csv('MISAMIS ORIENTAL.csv')

# Region 11

## Davao del Sur

#### Sergio-Loon Coffee Farm

In [323]:
dvo_sur_1 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao del Sur\Sergio-Loon Coffee Farm.csv",header = 2) 

In [324]:
dvo_sur_1 = extract_year_date(dvo_sur_1)

In [325]:
dvo_sur_1_dir = aggregate_wind_dir(dvo_sur_1)

In [326]:
dvo_sur_1_speed = aggregate_wind_speed(dvo_sur_1,dvo_sur_1_dir)

In [327]:
dvo_sur_1_wind= rescaled_wind(dvo_sur_1_dir,dvo_sur_1_speed)

In [328]:
dvo_sur_1_ave = get_mean(dvo_sur_1)

In [329]:
dvo_sur_1_sum = get_sum(dvo_sur_1)

In [330]:
dvo_sur_1_mode= get_mode(dvo_sur_1)

In [331]:
dvo_sur_1_not_wind = pd.concat([dvo_sur_1_ave, dvo_sur_1_sum,dvo_sur_1_mode],axis = 1)

In [332]:
dvo_sur_1_final = dvo_sur_1_wind.merge(dvo_sur_1_not_wind , how='inner',on='year')

#### Lendilou Loon Farm

In [333]:
dvo_sur_2 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao del Sur\Lendilou Loon Farm.csv",header = 2) 

In [336]:
dvo_sur_2= extract_year_date(dvo_sur_2)

In [337]:
dvo_sur_2_dir = aggregate_wind_dir(dvo_sur_2)

In [338]:
dvo_sur_2_speed = aggregate_wind_speed(dvo_sur_2,dvo_sur_2_dir)

In [339]:
dvo_sur_2_wind= rescaled_wind(dvo_sur_2_dir,dvo_sur_2_speed)

In [340]:
dvo_sur_2_ave = get_mean(dvo_sur_2)

In [341]:
dvo_sur_2_sum = get_sum(dvo_sur_2)

In [342]:
dvo_sur_2_mode = get_mode(dvo_sur_2)

In [343]:
dvo_sur_2_not_wind = pd.concat([dvo_sur_2_ave, dvo_sur_2_sum,dvo_sur_2_mode],axis = 1)

In [344]:
dvo_sur_2_final = dvo_sur_2_wind.merge(dvo_sur_2_not_wind , how='inner',on='year')

#### Payot Farm

In [345]:
dvo_sur_3 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao del Sur\Payot Farm.csv",header = 2) 

In [347]:
dvo_sur_3= extract_year_date(dvo_sur_3)

In [348]:
dvo_sur_3_dir = aggregate_wind_dir(dvo_sur_3)

In [349]:
dvo_sur_3_speed = aggregate_wind_speed(dvo_sur_3,dvo_sur_3_dir)

In [350]:
dvo_sur_3_wind= rescaled_wind(dvo_sur_3_dir,dvo_sur_3_speed)

In [351]:
dvo_sur_3_ave = get_mean(dvo_sur_3)

In [352]:
dvo_sur_3_sum = get_sum(dvo_sur_3)

In [353]:
dvo_sur_3_mode = get_mode(dvo_sur_3)

In [354]:
dvo_sur_3_not_wind = pd.concat([dvo_sur_3_ave, dvo_sur_3_sum,dvo_sur_3_mode],axis = 1)

In [355]:
dvo_sur_3_final = dvo_sur_3_wind.merge(dvo_sur_3_not_wind , how='inner',on='year')

#### Kim's Coffeee Farm

In [356]:
dvo_sur_4 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao del Sur\Kim's Coffee Farm.csv",header = 2) 

In [357]:
dvo_sur_4 = extract_year_date(dvo_sur_4)

In [358]:
dvo_sur_4_dir = aggregate_wind_dir(dvo_sur_4)

In [359]:
dvo_sur_4_speed = aggregate_wind_speed(dvo_sur_4,dvo_sur_4_dir)

In [360]:
dvo_sur_4_wind= rescaled_wind(dvo_sur_4_dir,dvo_sur_4_speed)

In [361]:
dvo_sur_4_ave = get_mean(dvo_sur_4)

In [362]:
dvo_sur_4_sum = get_sum(dvo_sur_4)

In [363]:
dvo_sur_4_mode = get_mode(dvo_sur_4)

In [364]:
dvo_sur_4_not_wind = pd.concat([dvo_sur_4_ave, dvo_sur_4_sum,dvo_sur_4_mode],axis = 1)

In [365]:
dvo_sur_4_final = dvo_sur_4_wind.merge(dvo_sur_4_not_wind , how='inner',on='year')

#### For Davao del Sur

In [366]:
dvo_sur = pd.concat([dvo_sur_1_final, dvo_sur_2_final, dvo_sur_3_final, dvo_sur_4_final],axis = 0)

In [367]:
regional_mode = region_mode(dvo_sur)

In [368]:
regional_mode = aggregate_region_windSpeed(dvo_sur,regional_mode)

In [369]:
regional_mean = region_mean(dvo_sur)

In [370]:
dvo_sur_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [371]:
dvo_sur_annual.insert(loc = 1, column= "Location", value = "DAVAO DEL SUR")

In [372]:
dvo_sur_annual.to_csv("DAVAO DEL SUR.csv")

## Davao City

In [130]:
dvo_city_1 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao City\gualberto farm.csv",header = 2) 

In [131]:
dvo_city_1  = extract_year_date(dvo_city_1)

In [132]:
dvo_city_1_dir = aggregate_wind_dir(dvo_city_1)

In [133]:
dvo_city_1_speed = aggregate_wind_speed(dvo_city_1,dvo_city_1_dir)

In [134]:
dvo_city_1_wind= rescaled_wind(dvo_city_1_dir,dvo_city_1_speed)

In [135]:
dvo_city_1_ave = get_mean(dvo_city_1)

In [136]:
dvo_city_1_sum = get_sum(dvo_city_1)

In [137]:
dvo_city_1_mode = get_mode(dvo_city_1)

In [138]:
dvo_city_1_not_wind = pd.concat([dvo_city_1_ave, dvo_city_1_sum,dvo_city_1_mode],axis = 1)

In [139]:
dvo_city_1_final = dvo_city_1_wind.merge(dvo_city_1_not_wind , how='inner',on='year')

In [140]:
dvo_city_1_final.insert(loc = 1, column= "Location", value = "DAVAO CITY")

In [141]:
dvo_city_1_final.to_csv('DAVAO CITY.csv')

## Davao del Norte


In [17]:
dvo_norte_1 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao del Norte\talaingod.csv",header = 2) 

In [19]:
dvo_norte_1  = extract_year_date(dvo_norte_1)

In [20]:
dvo_norte_1_dir = aggregate_wind_dir(dvo_norte_1)

In [21]:
dvo_norte_1_speed = aggregate_wind_speed(dvo_norte_1,dvo_norte_1_dir)

In [22]:
dvo_norte_1_wind= rescaled_wind(dvo_norte_1_dir,dvo_norte_1_speed)

In [23]:
dvo_norte_1_ave = get_mean(dvo_norte_1)

In [24]:
dvo_norte_1_sum= get_sum(dvo_norte_1)

In [25]:
dvo_norte_1_mode = get_mode(dvo_norte_1)

In [26]:
dvo_norte_1_not_wind = pd.concat([dvo_norte_1_ave, dvo_norte_1_sum,dvo_norte_1_mode],axis = 1)

In [27]:
dvo_norte_1_final = dvo_norte_1_wind.merge(dvo_norte_1_not_wind , how='inner',on='year')

In [28]:
dvo_norte_1_final.insert(loc = 1, column= "Location", value = "DAVAO DEL NORTE")

In [29]:
dvo_norte_1_final.to_csv('DAVAO DEL NORTE.csv')

## Davao Oriental

In [30]:
dvo_or_1 = pd.read_csv(r"D:\Thesis\Datasets\11 - Davao Oriental\tarragona.csv",header = 2) 

In [31]:
dvo_or_1 = extract_year_date(dvo_or_1)

In [32]:
dvo_or_1_dir = aggregate_wind_dir(dvo_or_1)

In [33]:
dvo_or_1_speed = aggregate_wind_speed(dvo_or_1,dvo_or_1_dir)

In [34]:
dvo_or_1_wind= rescaled_wind(dvo_or_1_dir,dvo_or_1_speed)

In [35]:
dvo_or_1_ave = get_mean(dvo_or_1)

In [36]:
dvo_or_1_sum= get_sum(dvo_or_1)

In [37]:
dvo_or_1_mode = get_mode(dvo_or_1)

In [38]:
dvo_or_1_not_wind = pd.concat([dvo_or_1_ave, dvo_or_1_sum,dvo_or_1_mode ],axis = 1)


In [39]:
dvo_or_1_final = dvo_or_1_wind.merge(dvo_or_1_not_wind , how='inner',on='year')

In [40]:
dvo_or_1_final.insert(loc = 1, column= "Location", value = "DAVAO ORIENTAL")

In [41]:
dvo_or_1_final.to_csv("DAVAO ORIENTAL.csv")

## Davao de Oro

In [42]:
dvo_de_oro_1 = pd.read_csv(r"D:\Thesis\Datasets\11- Davao de Oro\delio cezar.csv",header = 2) 

In [43]:
dvo_de_oro_1 = extract_year_date(dvo_de_oro_1)

In [44]:
dvo_de_oro_1_dir = aggregate_wind_dir(dvo_de_oro_1)

In [45]:
dvo_de_oro_1_speed = aggregate_wind_speed(dvo_de_oro_1,dvo_de_oro_1_dir)

In [46]:
dvo_de_oro_1_wind= rescaled_wind(dvo_de_oro_1_dir,dvo_de_oro_1_speed)

In [47]:
dvo_de_oro_1_ave = get_mean(dvo_de_oro_1)

In [48]:
dvo_de_oro_1_sum= get_sum(dvo_de_oro_1)

In [49]:
dvo_de_oro_1_mode= get_mode(dvo_de_oro_1)

In [50]:
dvo_de_oro_1_not_wind = pd.concat([dvo_de_oro_1_ave, dvo_de_oro_1_sum,dvo_de_oro_1_mode],axis = 1)

In [51]:
dvo_de_oro_1_final = dvo_de_oro_1_wind.merge(dvo_de_oro_1_not_wind , how='inner',on='year')

In [52]:
dvo_de_oro_1_final.insert(loc = 1, column= "Location", value ="DAVAO DE ORO")

In [53]:
dvo_de_oro_1_final.to_csv("DAVAO DE ORO.csv")

# Region 12

## Cotabato

#### Shiela Livera Leong

In [399]:
cotabato_1 = pd.read_csv(r"D:\Thesis\Datasets\12 - Cotabato\Sheila Livera Leong.csv",header = 2) 

In [400]:
cotabato_1 = extract_year_date(cotabato_1)

In [401]:
cotabato_1_dir = aggregate_wind_dir(cotabato_1)

In [402]:
cotabato_1_speed = aggregate_wind_speed(cotabato_1,cotabato_1_dir)

In [403]:
cotabato_1_wind= rescaled_wind(cotabato_1_dir,cotabato_1_speed)

In [404]:
cotabato_1_ave = get_mean(cotabato_1)

In [405]:
cotabato_1_sum = get_sum(cotabato_1)

In [406]:
cotabato_1_mode = get_mode(cotabato_1)

In [407]:
cotabato_1_not_wind = pd.concat([cotabato_1_ave, cotabato_1_sum,cotabato_1_mode],axis = 1)

In [408]:
cotabato_1_final = cotabato_1_wind.merge(cotabato_1_not_wind , how='inner',on='year')

#### Heaven's Bounty Farm

In [409]:
cotabato_2 = pd.read_csv(r"D:\Thesis\Datasets\12 - Cotabato\Heaven_s Bounty Farm.csv",header = 2) 

In [410]:
cotabato_2 = extract_year_date(cotabato_2)

In [411]:
cotabato_2_dir = aggregate_wind_dir(cotabato_2)

In [412]:
cotabato_2_speed = aggregate_wind_speed(cotabato_2 ,cotabato_2_dir)

In [413]:
cotabato_2_wind= rescaled_wind(cotabato_2_dir,cotabato_2_speed)

In [414]:
cotabato_2_ave = get_mean(cotabato_2)

In [415]:
cotabato_2_sum = get_sum(cotabato_2)

In [416]:
cotabato_2_mode = get_mode(cotabato_2)

In [417]:
cotabato_2_not_wind = pd.concat([cotabato_2_ave, cotabato_2_sum,cotabato_2_mode],axis = 1)

In [418]:
cotabato_2_final = cotabato_2_wind.merge(cotabato_2_not_wind , how='inner',on='year')

#### Kuvi Integrated Farm

In [419]:
cotabato_3 = pd.read_csv(r"D:\Thesis\Datasets\12 - Cotabato\Kuvi Integrated Farm.csv",header = 2) 

In [420]:
cotabato_3  = extract_year_date(cotabato_3)

In [421]:
cotabato_3_dir = aggregate_wind_dir(cotabato_3)

In [422]:
cotabato_3_speed = aggregate_wind_speed(cotabato_3,cotabato_3_dir)

In [423]:
cotabato_3_wind= rescaled_wind(cotabato_3_dir,cotabato_3_speed)

In [424]:
cotabato_3_ave = get_mean(cotabato_3)

In [425]:
cotabato_3_sum = get_sum(cotabato_3)

In [426]:
cotabato_3_mode = get_mode(cotabato_3)

In [427]:
cotabato_3_not_wind = pd.concat([cotabato_3_ave, cotabato_3_sum,cotabato_3_mode],axis = 1)

In [428]:
cotabato_3_final = cotabato_3_wind.merge(cotabato_3_not_wind , how='inner',on='year')

#### Sobee-It Farm

In [429]:
cotabato_4 = pd.read_csv(r"D:\Thesis\Datasets\12 - Cotabato\Sobee-It Farm.csv",header = 2) 

In [430]:
cotabato_4  = extract_year_date(cotabato_4)

In [431]:
cotabato_4_dir = aggregate_wind_dir(cotabato_4)

In [432]:
cotabato_4_speed = aggregate_wind_speed(cotabato_4,cotabato_4_dir)

In [433]:
cotabato_4_wind= rescaled_wind(cotabato_4_dir,cotabato_4_speed)

In [434]:
cotabato_4_ave = get_mean(cotabato_4)

In [435]:
cotabato_4_sum = get_sum(cotabato_4)

In [436]:
cotabato_4_mode = get_mode(cotabato_4)

In [437]:
cotabato_4_not_wind = pd.concat([cotabato_4_ave, cotabato_4_sum,cotabato_4_mode],axis = 1)

In [438]:
cotabato_4_final = cotabato_4_wind.merge(cotabato_4_not_wind , how='inner',on='year')

#### For Cotabato 

In [439]:
cotabato = pd.concat([cotabato_1_final,cotabato_2_final,cotabato_3_final,cotabato_4_final ],axis = 0)

In [440]:
regional_mode = region_mode(cotabato)

In [441]:
regional_mode = aggregate_region_windSpeed(cotabato,regional_mode)

In [442]:
regional_mean = region_mean(cotabato)

In [443]:
cotabato_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [444]:
cotabato_annual.insert(loc = 1, column= "Location", value = "COTABATO")

In [445]:
cotabato_annual.to_csv("COTABATO.csv")

## South Cotabato 

#### Ava Odal/Tucogai

In [446]:
south_cot_1 = pd.read_csv(r"D:\Thesis\Datasets\12 - South Cotabato\Ava OdalTucogai.csv",header = 2) 

In [447]:
south_cot_1 = extract_year_date(south_cot_1)

In [448]:
south_cot_1_dir = aggregate_wind_dir(south_cot_1)

In [449]:
south_cot_1_speed = aggregate_wind_speed(south_cot_1,south_cot_1_dir)

In [450]:
south_cot_1_wind= rescaled_wind(south_cot_1_dir,south_cot_1_speed)

In [451]:
south_cot_1_ave = get_mean(south_cot_1)

In [452]:
south_cot_1_sum = get_sum(south_cot_1)

In [453]:
south_cot_1_mode = get_mode(south_cot_1)

In [454]:
south_cot_1_not_wind = pd.concat([south_cot_1_ave, south_cot_1_sum,south_cot_1_mode],axis = 1)

In [455]:
south_cot_1_final = south_cot_1_wind.merge(south_cot_1_not_wind , how='inner',on='year')

#### Salcedo Farms

In [456]:
south_cot_2 = pd.read_csv(r"D:\Thesis\Datasets\12 - South Cotabato\SALCEDO FARMS.csv",header = 2) 

In [457]:
south_cot_2 = extract_year_date(south_cot_2)

In [458]:
south_cot_2_dir = aggregate_wind_dir(south_cot_2)

In [459]:
south_cot_2_speed = aggregate_wind_speed(south_cot_2,south_cot_2_dir)

In [460]:
south_cot_2_wind= rescaled_wind(south_cot_2_dir,south_cot_2_speed)


In [461]:
south_cot_2_ave = get_mean(south_cot_2)

In [462]:
south_cot_2_sum = get_sum(south_cot_2)

In [463]:
south_cot_2_mode = get_mode(south_cot_2)

In [464]:
south_cot_2_not_wind = pd.concat([south_cot_2_ave, south_cot_2_sum,south_cot_2_mode],axis = 1)

In [465]:
south_cot_2_final = south_cot_2_wind.merge(south_cot_2_not_wind , how='inner',on='year')

#### For South Cotabato

In [467]:
south_cot = pd.concat([south_cot_1_final, south_cot_2_final],axis = 0)

In [468]:
regional_mode = region_mode(south_cot)


In [469]:
regional_mode = aggregate_region_windSpeed(south_cot,regional_mode)

In [470]:
regional_mean = region_mean(south_cot)

In [471]:
south_cot_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [472]:
south_cot_annual.insert(loc = 1, column= "Location", value = "SOUTH COTABATO")

In [474]:
south_cot_annual.to_csv("SOUTH COTABATO.csv")

## Sultan Kudarat

#### Carlo Posadas

In [476]:
sultan_kudarat_1 = pd.read_csv(r"D:\Thesis\Datasets\12 - Sultan Kudarat\Carlo Posadas.csv",header = 2) 

In [477]:
sultan_kudarat_1 = extract_year_date(sultan_kudarat_1)

In [478]:
sultan_kudarat_1_dir = aggregate_wind_dir(sultan_kudarat_1)

In [479]:
sultan_kudarat_1_speed = aggregate_wind_speed(sultan_kudarat_1,sultan_kudarat_1_dir)

In [480]:
sultan_kudarat_1_wind= rescaled_wind(sultan_kudarat_1_dir,sultan_kudarat_1_speed)

In [481]:
sultan_kudarat_1_ave = get_mean(sultan_kudarat_1)

In [482]:
sultan_kudarat_1_sum = get_sum(sultan_kudarat_1)

In [483]:
sultan_kudarat_1_mode = get_mode(sultan_kudarat_1)

In [484]:
sultan_kudarat_1_not_wind = pd.concat([sultan_kudarat_1_ave, sultan_kudarat_1_sum,sultan_kudarat_1_mode],axis = 1)

In [485]:
sultan_kudarat_1_final = sultan_kudarat_1_wind.merge(sultan_kudarat_1_not_wind , how='inner',on='year')

#### Kape Dulangan - SKVCI

In [486]:
sultan_kudarat_2 = pd.read_csv(r"D:\Thesis\Datasets\12 - Sultan Kudarat\Kape Dulangan - SKCVI.csv",header = 2) 

In [487]:
sultan_kudarat_2 = extract_year_date(sultan_kudarat_2)

In [488]:
sultan_kudarat_2_dir = aggregate_wind_dir(sultan_kudarat_2)

In [489]:
sultan_kudarat_2_speed = aggregate_wind_speed(sultan_kudarat_2,sultan_kudarat_2_dir)

In [490]:
sultan_kudarat_2_wind= rescaled_wind(sultan_kudarat_2_dir,sultan_kudarat_2_speed)

In [491]:
sultan_kudarat_2_ave = get_mean(sultan_kudarat_2)

In [492]:
sultan_kudarat_2_sum = get_sum(sultan_kudarat_2)

In [493]:
sultan_kudarat_2_mode = get_mode(sultan_kudarat_2)

In [494]:
sultan_kudarat_2_not_wind = pd.concat([sultan_kudarat_2_ave, sultan_kudarat_2_sum,sultan_kudarat_2_mode],axis = 1)

In [495]:
sultan_kudarat_2_final = sultan_kudarat_2_wind.merge(sultan_kudarat_2_not_wind , how='inner',on='year')

#### Jennifer Farillon

In [496]:
sultan_kudarat_3 = pd.read_csv(r"D:\Thesis\Datasets\12 - Sultan Kudarat\Jennifer Farillon.csv",header = 2) 

In [497]:
sultan_kudarat_3 = extract_year_date(sultan_kudarat_3)

In [498]:
sultan_kudarat_3_dir = aggregate_wind_dir(sultan_kudarat_3)

In [499]:
sultan_kudarat_3_speed = aggregate_wind_speed(sultan_kudarat_3,sultan_kudarat_3_dir)

In [500]:
sultan_kudarat_3_wind= rescaled_wind(sultan_kudarat_3_dir,sultan_kudarat_3_speed)

In [501]:
sultan_kudarat_3_ave = get_mean(sultan_kudarat_3)

In [502]:
sultan_kudarat_3_sum= get_sum(sultan_kudarat_3)

In [503]:
sultan_kudarat_3_mode= get_mode(sultan_kudarat_3)

In [504]:
sultan_kudarat_3_not_wind = pd.concat([sultan_kudarat_3_ave, sultan_kudarat_3_sum,sultan_kudarat_3_mode],axis = 1)

In [505]:
sultan_kudarat_3_final = sultan_kudarat_3_wind.merge(sultan_kudarat_3_not_wind , how='inner',on='year')

#### Kalanawan Rainforest Coffee

In [507]:
sultan_kudarat_4 = pd.read_csv(r"D:\Thesis\Datasets\12 - Sultan Kudarat\Kalanawan Rainforest Coffee.csv",header = 2) 

In [508]:
sultan_kudarat_4 = extract_year_date(sultan_kudarat_4)

In [509]:
sultan_kudarat_4_dir = aggregate_wind_dir(sultan_kudarat_4)

In [510]:
sultan_kudarat_4_speed = aggregate_wind_speed(sultan_kudarat_4,sultan_kudarat_4_dir)

In [511]:
sultan_kudarat_4_wind= rescaled_wind(sultan_kudarat_4_dir,sultan_kudarat_4_speed)


In [512]:
sultan_kudarat_4_ave = get_mean(sultan_kudarat_4)

In [513]:
sultan_kudarat_4_sum= get_sum(sultan_kudarat_4)

In [514]:
sultan_kudarat_4_mode= get_mode(sultan_kudarat_4)

In [515]:
sultan_kudarat_4_not_wind = pd.concat([sultan_kudarat_4_ave, sultan_kudarat_4_sum,sultan_kudarat_4_mode],axis = 1)

In [516]:
sultan_kudarat_4_final = sultan_kudarat_4_wind.merge(sultan_kudarat_4_not_wind , how='inner',on='year')

#### For Sultan Kudarat

In [518]:
sultan_kudarat = pd.concat([sultan_kudarat_1_final,sultan_kudarat_2_final, sultan_kudarat_3_final, sultan_kudarat_4_final],axis = 0)

In [519]:
regional_mode = region_mode(sultan_kudarat)

In [520]:
regional_mode = aggregate_region_windSpeed(sultan_kudarat,regional_mode)

In [521]:
regional_mean = region_mean(sultan_kudarat)

In [522]:
sultan_kudarat_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [523]:
sultan_kudarat_annual.insert(loc = 1, column= "Location", value = "SULTAN KUDARAT")

In [524]:
sultan_kudarat_annual.to_csv('SULTAN KUDARAT.csv')

## Sarangani 

### Malalag Cogon

In [54]:
sarangani_1 = pd.read_csv(r"D:\Thesis\Datasets\12 - Sarangani\malalag cogon.csv",header = 2) 

In [55]:
sarangani_1 = extract_year_date(sarangani_1 )

In [56]:
sarangani_1_dir = aggregate_wind_dir(sarangani_1 )

In [57]:
sarangani_1_speed = aggregate_wind_speed(sarangani_1,sarangani_1_dir)

In [58]:
sarangani_1_wind= rescaled_wind(sarangani_1_dir,sarangani_1_speed)

In [60]:
sarangani_1_ave = get_mean(sarangani_1)

In [61]:
sarangani_1_sum = get_sum(sarangani_1)

In [70]:
sarangani_1_mode= get_mode(sarangani_1)

In [71]:
sarangani_1_not_wind = pd.concat([sarangani_1_ave, sarangani_1_sum,sarangani_1_mode],axis = 1)

In [74]:
sarangani_1_final = sarangani_1_wind.merge(sarangani_1_not_wind , how='inner',on='year')

### Lower Mainit

In [75]:
sarangani_2 = pd.read_csv(r"D:\Thesis\Datasets\12 - Sarangani\lower mainit.csv",header = 2) 

In [76]:
sarangani_2 = extract_year_date(sarangani_2)

In [77]:
sarangani_2_dir = aggregate_wind_dir(sarangani_2)

In [78]:
sarangani_2_speed = aggregate_wind_speed(sarangani_2,sarangani_2_dir)

In [79]:
sarangani_2_wind= rescaled_wind(sarangani_2_dir,sarangani_2_speed)

In [80]:
sarangani_2_ave = get_mean(sarangani_2)

In [81]:
sarangani_2_sum = get_sum(sarangani_2)

In [82]:
sarangani_2_mode = get_mode(sarangani_2)

In [83]:
sarangani_2_not_wind = pd.concat([sarangani_2_ave, sarangani_2_sum,sarangani_2_mode],axis = 1)

In [84]:
sarangani_2_final = sarangani_2_wind.merge(sarangani_2_not_wind , how='inner',on='year')

### For Sarangani

In [85]:
sarangani = pd.concat([sarangani_1_final,sarangani_2_final],axis = 0)

In [86]:
regional_mode = region_mode(sarangani)

In [87]:
regional_mode = aggregate_region_windSpeed(sarangani,regional_mode)

In [88]:
regional_mean = region_mean(sarangani)

In [89]:
sarangani_annual = regional_mode.merge(regional_mean , how='inner',on='year')

In [90]:
sarangani_annual.insert(loc = 1, column= "Location", value = "SARANGANI")

In [91]:
sarangani_annual.to_csv('SARANGANI.csv')

# Region 13

## Agusan del Norte

In [525]:
agusan_norte_1 = pd.read_csv(r"D:\Thesis\Datasets\13 - Agusan del Norte\Casiklan, Las Nieves.csv",header = 2) 

In [526]:
agusan_norte_1 = extract_year_date(agusan_norte_1)

In [527]:
agusan_norte_1_dir = aggregate_wind_dir(agusan_norte_1)

In [528]:
agusan_norte_1_speed = aggregate_wind_speed(agusan_norte_1,agusan_norte_1_dir)

In [529]:
agusan_norte_1_wind= rescaled_wind(agusan_norte_1_dir,agusan_norte_1_speed)

In [530]:
agusan_norte_1_ave = get_mean(agusan_norte_1)

In [531]:
agusan_norte_1_sum = get_sum(agusan_norte_1)

In [532]:
agusan_norte_1_mode = get_mode(agusan_norte_1)

In [533]:
agusan_norte_1_not_wind = pd.concat([agusan_norte_1_ave, agusan_norte_1_sum,agusan_norte_1_mode],axis = 1)

In [534]:
agusan_norte_1_final = agusan_norte_1_wind.merge(agusan_norte_1_not_wind , how='inner',on='year')

In [535]:
agusan_norte_1_final.insert(loc = 1, column= "Location", value = 'AGUSAN DEL NORTE')

In [536]:
agusan_norte_1_final.to_csv("AGUSAN DEL NORTE.csv")

## Agusan del Sur

In [537]:
agusan_sur_1 = pd.read_csv(r"D:\Thesis\Datasets\13 - Agusan del Sur\Rankert's Coffee Farm.csv",header = 2) 

In [538]:
agusan_sur_1 = extract_year_date(agusan_sur_1)

In [539]:
agusan_sur_1_dir = aggregate_wind_dir(agusan_sur_1)

In [540]:
agusan_sur_1_speed = aggregate_wind_speed(agusan_sur_1,agusan_sur_1_dir)

In [541]:
agusan_sur_1_wind= rescaled_wind(agusan_sur_1_dir,agusan_sur_1_speed)

In [542]:
agusan_sur_1_ave = get_mean(agusan_sur_1)

In [543]:
agusan_sur_1_sum = get_sum(agusan_sur_1)

In [544]:
agusan_sur_1_mode = get_mode(agusan_sur_1)

In [545]:
agusan_sur_1_not_wind = pd.concat([agusan_sur_1_ave, agusan_sur_1_sum,agusan_sur_1_mode],axis = 1)

In [546]:
agusan_sur_1_final = agusan_sur_1_wind.merge(agusan_sur_1_not_wind , how='inner',on='year')

In [547]:
agusan_sur_1_final.insert(loc = 1, column= "Location", value = "AGUSAN DEL SUR")

In [548]:
agusan_sur_1_final.to_csv("AGUSAN DEL SUR.csv")

## Surigao del Sur

In [105]:
surigao_sur_1 = pd.read_csv(r"D:\Thesis\Datasets\13 - Surigao del Sur\Amparitas Integrated Natural Farm.csv",header = 2) 

In [106]:
surigao_sur_1= extract_year_date(surigao_sur_1)

In [107]:
surigao_sur_1_dir = aggregate_wind_dir(surigao_sur_1)

In [108]:
surigao_sur_1_speed = aggregate_wind_speed(surigao_sur_1,surigao_sur_1_dir)

In [109]:
surigao_sur_1_wind= rescaled_wind(surigao_sur_1_dir,surigao_sur_1_speed)

In [110]:
surigao_sur_1_ave = get_mean(surigao_sur_1)


In [111]:
surigao_sur_1_sum = get_sum(surigao_sur_1)

In [112]:
surigao_sur_1_mode = get_mode(surigao_sur_1)

In [113]:
surigao_sur_1_not_wind = pd.concat([surigao_sur_1_ave, surigao_sur_1_sum,surigao_sur_1_mode],axis = 1)

In [114]:
surigao_sur_1_final = surigao_sur_1_wind.merge(surigao_sur_1_not_wind , how='inner',on='year')

In [115]:
surigao_sur_1_final.insert(loc = 1, column= "Location", value = "SURIGAO DEL SUR")

In [116]:
surigao_sur_1_final.to_csv("SURIGAO DEL SUR.csv")

# BARMM

## Basilan

In [118]:
basilan_1 = pd.read_csv(r"D:\Thesis\Datasets\BARMM - Basilan\isabela.csv",header = 2) 

In [119]:
basilan_1= extract_year_date(basilan_1)

In [120]:
basilan_1_dir = aggregate_wind_dir(basilan_1)

In [121]:
basilan_1_speed = aggregate_wind_speed(basilan_1,basilan_1_dir)

In [122]:
basilan_1_wind= rescaled_wind(basilan_1_dir,basilan_1_speed)

In [123]:
basilan_1_ave = get_mean(basilan_1)

In [124]:
basilan_1_sum = get_sum(basilan_1)

In [125]:
basilan_1_mode = get_mode(basilan_1)

In [126]:
basilan_1_not_wind = pd.concat([basilan_1_ave, basilan_1_sum,basilan_1_mode],axis = 1)

In [127]:
basilan_1_final = basilan_1_wind.merge(basilan_1_not_wind , how='inner',on='year')


In [128]:
basilan_1_final.insert(loc = 1, column= "Location", value = 'BASILAN')

In [129]:
basilan_1_final.to_csv("BASILAN.csv")

## Lanao del Sur

In [9]:
lanao_sur_1 = pd.read_csv(r"D:\Thesis\Datasets\BARMM - Lanao del Sur\TOCAO Agri-farm.csv",header = 2) 

In [11]:
lanao_sur_1 = extract_year_date(lanao_sur_1)

In [12]:
lanao_sur_1_dir = aggregate_wind_dir(lanao_sur_1)

In [13]:
lanao_sur_1_speed = aggregate_wind_speed(lanao_sur_1,lanao_sur_1_dir)

In [14]:
lanao_sur_1_wind= rescaled_wind(lanao_sur_1_dir,lanao_sur_1_speed)

In [23]:
lanao_sur_1_ave = get_mean(lanao_sur_1)

In [24]:
lanao_sur_1_sum = get_sum(lanao_sur_1)

In [25]:
lanao_sur_1_mode = get_mode(lanao_sur_1)

In [26]:
lanao_sur_1_not_wind = pd.concat([lanao_sur_1_ave, lanao_sur_1_sum,lanao_sur_1_mode],axis = 1)

In [27]:
lanao_sur_1_final = lanao_sur_1_wind.merge(lanao_sur_1_not_wind , how='inner',on='year')

In [28]:
lanao_sur_1_final.insert(loc = 1, column= "Location", value = "LANAO DEL SUR")

In [29]:
lanao_sur_1_final.to_csv('LANAO DEL SUR.csv')

## Maguindanao 

In [30]:
maguindanao_1 = pd.read_csv(r"D:\Thesis\Datasets\BARMM - Maguindanao\South Upi, Maguindanao.csv",header = 2) 

In [31]:
maguindanao_1 = extract_year_date(maguindanao_1)

In [32]:
maguindanao_1_dir = aggregate_wind_dir(maguindanao_1)

In [33]:
maguindanao_1_speed = aggregate_wind_speed(maguindanao_1,maguindanao_1_dir)

In [34]:
maguindanao_1_wind= rescaled_wind(maguindanao_1_dir,maguindanao_1_speed)

In [35]:
maguindanao_1_ave = get_mean(maguindanao_1)

In [36]:
maguindanao_1_sum = get_sum(maguindanao_1)

In [37]:
maguindanao_1_mode = get_mode(maguindanao_1)

In [38]:
maguindanao_1_not_wind = pd.concat([maguindanao_1_ave, maguindanao_1_sum,maguindanao_1_mode],axis = 1)

In [39]:
maguindanao_1_final = maguindanao_1_wind.merge(maguindanao_1_not_wind , how='inner',on='year')

In [40]:
maguindanao_1_final.insert(loc = 1, column= "Location", value = "MAGUINDANAO")

## Sulu

In [42]:
sulu_1 = pd.read_csv(r"D:\Thesis\Datasets\BARMM - Sulu\Dennis Coffee.csv",header = 2) 

In [43]:
sulu_1= extract_year_date(sulu_1)

In [44]:
sulu_1_dir = aggregate_wind_dir(sulu_1)

In [45]:
sulu_1_speed = aggregate_wind_speed(sulu_1,sulu_1_dir )

In [46]:
sulu_1_wind= rescaled_wind(sulu_1_dir,sulu_1_speed)

In [47]:
sulu_1_ave = get_mean(sulu_1)

In [48]:
sulu_1_sum = get_sum(sulu_1)

In [49]:
sulu_1_mode = get_mode(sulu_1)

In [50]:
sulu_1_not_wind = pd.concat([sulu_1_ave, sulu_1_sum,sulu_1_mode],axis = 1)

In [51]:
sulu_1_final = sulu_1_wind.merge(sulu_1_not_wind , how='inner',on='year')

In [52]:
sulu_1_final.insert(loc = 1, column= "Location", value = "SULU")

In [53]:
sulu_1_final.to_csv("SULU.csv")